In [1]:
# Download today puzzle & input
!aoc --version
!aoc download -i input.txt --overwrite -p README.md --year 2022 --day 05

aoc-cli 0.6.0
Loaded session cookie from "/home/kev/.adventofcode.session".
Fetching puzzle for day 5, 2022...
Saving puzzle description to "README.md"...
Saving puzzle input to "input.txt"...
Done!


# --- Day 5: Supply Stacks ---
----------

The expedition can depart as soon as the final supplies have been unloaded from the ships. Supplies are stored in stacks of marked *crates*, but because the needed supplies are buried under many other crates, the crates need to be rearranged.

The ship has a *giant cargo crane* capable of moving crates between stacks. To ensure none of the crates get crushed or fall over, the crane operator will rearrange them in a series of carefully-planned steps. After the crates are rearranged, the desired crates will be at the top of each stack.

The Elves don't want to interrupt the crane operator during this delicate procedure, but they forgot to ask her *which* crate will end up where, and they want to be ready to unload them as soon as possible so they can embark.

They do, however, have a drawing of the starting stacks of crates *and* the rearrangement procedure (your puzzle input). For example:

```
    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2

```

In this example, there are three stacks of crates. Stack 1 contains two crates: crate `Z` is on the bottom, and crate `N` is on top. Stack 2 contains three crates; from bottom to top, they are crates `M`, `C`, and `D`. Finally, stack 3 contains a single crate, `P`.

Then, the rearrangement procedure is given. In each step of the procedure, a quantity of crates is moved from one stack to a different stack. In the first step of the above rearrangement procedure, one crate is moved from stack 2 to stack 1, resulting in this configuration:

```
[D]        
[N] [C]    
[Z] [M] [P]
 1   2   3

```

In the second step, three crates are moved from stack 1 to stack 3. Crates are moved *one at a time*, so the first crate to be moved (`D`) ends up below the second and third crates:

```
        [Z]
        [N]
    [C] [D]
    [M] [P]
 1   2   3

```

Then, both crates are moved from stack 2 to stack 1. Again, because crates are moved *one at a time*, crate `C` ends up below crate `M`:

```
        [Z]
        [N]
[M]     [D]
[C]     [P]
 1   2   3

```

Finally, one crate is moved from stack 1 to stack 2:

```
        [Z]
        [N]
        [D]
[C] [M] [P]
 1   2   3

```

The Elves just need to know *which crate will end up on top of each stack*; in this example, the top crates are `C` in stack 1, `M` in stack 2, and `Z` in stack 3, so you should combine these together and give the Elves the message `*CMZ*`.

*After the rearrangement procedure completes, what crate ends up on top of each stack?*


In [2]:
from pprint import pprint 

with open('input.txt', 'rt') as f:
    lines = [_.strip() for _ in f.readlines()]

# Split at empty line
split = lines.index('')
stack = lines[:split]
movements = lines[split+1:]


In [3]:
# validation parse
stack

['[N]         [C]     [Z]',
 '[Q] [G]     [V]     [S]         [V]',
 '[L] [C]     [M]     [T]     [W] [L]',
 '[S] [H]     [L]     [C] [D] [H] [S]',
 '[C] [V] [F] [D]     [D] [B] [Q] [F]',
 '[Z] [T] [Z] [T] [C] [J] [G] [S] [Q]',
 '[P] [P] [C] [W] [W] [F] [W] [J] [C]',
 '[T] [L] [D] [G] [P] [P] [V] [N] [R]',
 '1   2   3   4   5   6   7   8   9']

In [5]:
def parse_stack(lines):
    """Parse a list of plain text lines into a list-of-lists

    Example:

        [D]    
    [N] [C]    
    [Z] [M] [P]
    1   2   3

    Args:
        lines (_type_): Pretty formatted lines

    Returns:
        int: List-of-lists representing stacks in lowest-highest order. Ie, Highest box is last element in each list.
    """
    stacks = []
    for line in lines:
        stack_i = 0
        for i in range(1, len(line), 4):
            stack_i += 1
            if len(stacks) < stack_i:
                stacks.append([])
            if line[i] not in ['', ' ']:
                stacks[int((i-1)/4)].insert(0, line[i])
    return stacks

# Preview
pprint(parse_stack(stack))

[['T', 'P', 'Z', 'C', 'S', 'L', 'Q', 'N'],
 ['L', 'P', 'T', 'V', 'H', 'C', 'G'],
 ['D', 'C', 'Z', 'F'],
 ['G', 'W', 'T', 'D', 'L', 'M', 'V', 'C'],
 ['P', 'W', 'C'],
 ['P', 'F', 'J', 'D', 'C', 'T', 'S', 'Z'],
 ['V', 'W', 'G', 'B', 'D'],
 ['N', 'J', 'S', 'Q', 'H', 'W'],
 ['R', 'C', 'Q', 'F', 'S', 'L', 'V']]


In [4]:
# Preview 
movements[:6]

['move 6 from 2 to 1',
 'move 4 from 6 to 3',
 'move 1 from 6 to 5',
 'move 8 from 3 to 8',
 'move 13 from 8 to 2',
 'move 2 from 7 to 6']

In [10]:
def perform_movement_9000(stacks, action: str, verbose: bool = False):
    """Perform part 1 movement on stacks from plain-text action

    move 1 from 2 to 1
    move 3 from 1 to 3
    move 2 from 2 to 1

    Args:
        stacks (list[int]): List-of-lists representing stacks
        action (str): Plain text movement.

    Returns:
        list[int]: Updates stacks
    """
    _, count, _, from_stack, _, to_stack = action.split()
    count = int(count)
    from_stack = int(from_stack) - 1
    to_stack = int(to_stack) - 1
    
    if verbose: 
        print(f"{action} count: {count} from: {from_stack} to: {to_stack}")
        print(f"before from: {stacks[from_stack]} to: {stacks[to_stack]}")

    for i in range(0, count):
        box = stacks[from_stack].pop()
        stacks[to_stack].append(box)

    if verbose: 
        print(f"after  from: {stacks[from_stack]} to: {stacks[to_stack]}")

    return stacks

# Verify logic
_ = perform_movement_9000(parse_stack(stack), movements[0], verbose=True)

move 6 from 2 to 1 count: 6 from: 1 to: 0
before from: ['L', 'P', 'T', 'V', 'H', 'C', 'G'] to: ['T', 'P', 'Z', 'C', 'S', 'L', 'Q', 'N']
after  from: ['L'] to: ['T', 'P', 'Z', 'C', 'S', 'L', 'Q', 'N', 'G', 'C', 'H', 'V', 'T', 'P']


[['T', 'P', 'Z', 'C', 'S', 'L', 'Q', 'N', 'G', 'C', 'H', 'V', 'T', 'P'],
 ['L'],
 ['D', 'C', 'Z', 'F'],
 ['G', 'W', 'T', 'D', 'L', 'M', 'V', 'C'],
 ['P', 'W', 'C'],
 ['P', 'F', 'J', 'D', 'C', 'T', 'S', 'Z'],
 ['V', 'W', 'G', 'B', 'D'],
 ['N', 'J', 'S', 'Q', 'H', 'W'],
 ['R', 'C', 'Q', 'F', 'S', 'L', 'V']]

In [7]:
# Initialise stacks from input
stacks = parse_stack(stack)

# Perform each movement
for movement in movements:
    stacks = perform_movement_9000(stacks, movement)

# Get top (last) box in each stack
print("answer 1:", ''.join([s.pop() for s in stacks]))

answer 1: SVFDLGLWV


\--- Part Two ---
----------

As you watch the crane operator expertly rearrange the crates, you notice the process isn't following your prediction.

Some mud was covering the writing on the side of the crane, and you quickly wipe it away. The crane isn't a CrateMover 9000 - it's a *CrateMover 9001*.

The CrateMover 9001 is notable for many new and exciting features: air conditioning, leather seats, an extra cup holder, and *the ability to pick up and move multiple crates at once*.

Again considering the example above, the crates begin in the same configuration:

```
    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3

```

Moving a single crate from stack 2 to stack 1 behaves the same as before:

```
[D]        
[N] [C]    
[Z] [M] [P]
 1   2   3

```

However, the action of moving three crates from stack 1 to stack 3 means that those three moved crates *stay in the same order*, resulting in this new configuration:

```
        [D]
        [N]
    [C] [Z]
    [M] [P]
 1   2   3

```

Next, as both crates are moved from stack 2 to stack 1, they *retain their order* as well:

```
        [D]
        [N]
[C]     [Z]
[M]     [P]
 1   2   3

```

Finally, a single crate is still moved from stack 1 to stack 2, but now it's crate `C` that gets moved:

```
        [D]
        [N]
        [Z]
[M] [C] [P]
 1   2   3

```

In this example, the CrateMover 9001 has put the crates in a totally different order: `*MCD*`.

Before the rearrangement process finishes, update your simulation so that the Elves know where they should stand to be ready to unload the final supplies. *After the rearrangement procedure completes, what crate ends up on top of each stack?*


In [12]:
def perform_movement_9001(stacks, action, verbose: bool = False):
    """Perform part-2 movement on stacks from plain-text action.

    Each action is a single movement and maintains the order of boxes.

    move 1 from 2 to 1
    move 3 from 1 to 3
    move 2 from 2 to 1

    Args:
        stacks (list[int]): List-of-lists representing stacks
        action (str): Plain text movement.

    Returns:
        list[int]: Updates stacks
    """
    _, count, _, from_stack, _, to_stack = action.split()
    count = int(count)
    from_stack = int(from_stack) - 1
    to_stack = int(to_stack) - 1

    if verbose: 
        print(f"{action} count: {count} from: {from_stack} to: {to_stack}")
        print(f"before from: {stacks[from_stack]} to: {stacks[to_stack]}")

    boxes = stacks[from_stack][0-count:]
    stacks[from_stack] = stacks[from_stack][:0-count]
    stacks[to_stack] += (boxes)
    
    if verbose: 
        print(f"after  from: {stacks[from_stack]} to: {stacks[to_stack]}")
    
    return stacks

# Verify logic
_ = perform_movement_9001(parse_stack(stack), movements[0], verbose=True)

move 6 from 2 to 1 count: 6 from: 1 to: 0
before from: ['L', 'P', 'T', 'V', 'H', 'C', 'G'] to: ['T', 'P', 'Z', 'C', 'S', 'L', 'Q', 'N']
after  from: ['L'] to: ['T', 'P', 'Z', 'C', 'S', 'L', 'Q', 'N', 'P', 'T', 'V', 'H', 'C', 'G']


In [9]:
# Initialise stacks from input
stacks = parse_stack(stack)

# Perform each movement
for movement in movements:
    stacks = perform_movement_9001(stacks, movement)

# Get top (last) box in each stack
print("answer 2:", ''.join([s.pop() for s in stacks]))

answer 2: DCVTCVPCL
